In [1]:
from modules.build_substring import NERDString

In [2]:
s = '''
    scientist Ryan Willett who attended Columbia University  after 2003-02-11 pdf docx txt
''' 

In [3]:
ns = NERDString(s)

In [4]:
ns.doc


    scientist Ryan Willett who attended Columbia University  after 2003-02-11 pdf docx txt

In [5]:
ns.entities

,name,entity
0,Ryan Willett,PERSON
1,Columbia University,ORG
2,2003-02-11,DATE


In [6]:
ns.data

{'persons': ['Ryan Willett'],
 'orgs': ['Columbia University'],
 'gpe': [],
 'min_date': '2003-02-11',
 'max_date': '2003-02-11',
 'urls': [],
 'filetypes': ['pdf', ' docx', 'txt']}

In [7]:
ns.text

'\n    scientist Ryan Willett who attended Columbia University  after 2003-02-11 pdf docx txt\n'

In [8]:
fromfrontend = {
  "end_date": "2023-01-30",
  "filetypes": [],
  "root_terms": [
    "Ryan Willett",
    " Columbia University",
    " scientist"
  ],
  "start_date": "2021-03-23"
}
fromfrontend

{'end_date': '2023-01-30',
 'filetypes': [],
 'root_terms': ['Ryan Willett', ' Columbia University', ' scientist'],
 'start_date': '2021-03-23'}

In [9]:
roots = ' '.join(fromfrontend['root_terms'])
roots

'Ryan Willett  Columbia University  scientist'

In [10]:
ns = NERDString(roots)

In [11]:
ns.data

{'persons': ['Ryan Willett'],
 'orgs': ['Columbia University'],
 'gpe': [],
 'min_date': '',
 'max_date': '',
 'urls': [],
 'filetypes': []}